#1 download audio and subtitles from youtube   'YOUTUBE DL' required
#2 extracting audio chunks of desired duration and sentences from subtitle and audio file downloaded earlier
#3 converting audio and subtitles to prescribed format 'SOX' required
#4 creating transcription file

In [2]:
#importing libraries
import os  
from pydub import AudioSegment
import lxml
from lxml import etree
import urllib.request
import re
import os.path
from os import path

In [11]:
!pip3 install pydub
!pip3 install lxml
!pip3 install pytube
!pip3 install certifi
!brew install youtube-dl

You are using pip version 19.0.3, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.3, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.3, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.3, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [16]:
os.system

<function posix.system(command)>

In [34]:
import ssl
#url_list is a list of url we want to work on
url_list=['https://www.youtube.com/watch?v=kcW4ABcY3zI','https://www.youtube.com/watch?v=0NV1KdWRHck&list=WL&index=4&t=0s', 'https://www.youtube.com/watch?v=YDp3EE0AM0s']

path='/Users/anshul/Desktop/ISTEM/azure-speech-testing/'
duration=180 #duration of audio chunks (sec)
duration=duration*1000

for url in url_list:
    # downloading the audio and subtitle of a youtube video
    gcontext = ssl.SSLContext()  # Only for gangstars
    youtube = etree.HTML(urllib.request.urlopen(url,context=gcontext).read()) 
    video_title = ''.join(youtube.xpath("//span[@id='eow-title']/@title")) #downloading the title from url
    video_title=(re.sub(' +', ' ',re.sub(r'[^\w\s]','',video_title) )).replace(' ','_')
    audio='youtube-dl --extract-audio --audio-format mp3 --output "'+video_title+'.mp3" '+url
    subtitle='youtube-dl --write-sub --sub-lang en --skip-download --output "'+video_title+'" '+url
    #downloading the audio from url
    if not os.path.exists(path+video_title+'.mp3'):print(os.system(audio))
    #downloading the audio from subtitle
    print(subtitle)
    if not os.path.exists(path+video_title+'.en.vtt'):print(os.system(subtitle))
    
    #print(audio)
    # creating transcribed text and audio chunks
    vid=video_title
    vid=vid.replace('? | ',' _ ').replace(' | ',' _ ')
    w=open(path+vid+'.en.vtt','r', encoding="utf8")
    x=w.readlines()
    w.close()

    #converting string of subtitle to time in ms
    def time_in_ms(source):
        q=(int)(source.split(".")[1])
        source=source.split(".")[0]
        tokens = source.split(":")
        secondsToMs = int(tokens[2]) * 1000
        minutesToMs = int(tokens[1]) * 60000
        hoursToMs = int(tokens[0]) * 3600000
        total = secondsToMs + minutesToMs + hoursToMs+q
        return (total)

    #parsing the subtitle file
    time_ms=[]
    sentence=[]
    for count,ele in enumerate(x): 
        curr=''
        if ele.startswith('00:'): #using 'OO:' to identify start of line of subtitle
            time_ms.append(time_in_ms(ele[:12]))
            count=count+1
            while(x[count]!='\n'):
                curr=curr+' '+(x[count][:-1])
                count=count+1
            sentence.append(curr)   
    time_ms = list(map(int, time_ms)) 
    len(time_ms)

    #creating chunks from subtitle
    text=[]
    start=time_ms[0]
    point=0
    start_t=[]
    for i,ele in enumerate(time_ms):
        if ele-start >= duration:
            end=time_ms[i-1]
            st=''
            while point<i-1:
                st=st+sentence[point]
                point=point+1
            text.append(st)
            start_t.append(start)
            start=end
            
    print('slicing')
    # slicing the audio and converting into desired format (.wav sampled at 16k Hz)               
    start_t.append(end)
    sound = AudioSegment.from_file(path+vid+'.mp3') 
    f= open(path+vid+"_transcribed.txt","w+") #creating trancription file
    for i in range(len(text)-1):
        speech=sound[start_t[i]:start_t[i+1]]
        name=vid+"t"+str(i)
        speech.export(path+name+".mp3")
        cmd="sox "+path+name+".mp3 -b 16 -e signed-integer -c 1 -r 16k -t wav "+path+name+".wav"
        os.system(cmd)
        os.remove(path+name+".mp3")
        tex=str((text[i].replace('’','\'').replace('…','.')).encode('ascii',"ignore"))[2:-1] # text cleaning (microsoft described format)
        f.write(str(i)+'\t'+tex[1:]+'\n')
    f.close()

youtube-dl --write-sub --sub-lang en --skip-download --output "A_well_educated_mind_vs_a_well_formed_mind_Dr_Shashi_Tharoor_at_TEDxGateway_2013" https://www.youtube.com/watch?v=kcW4ABcY3zI
slicing
youtube-dl --write-sub --sub-lang en --skip-download --output "Thoughts_on_humanity_fame_and_love_Shah_Rukh_Khan" https://www.youtube.com/watch?v=0NV1KdWRHck&list=WL&index=4&t=0s
slicing
youtube-dl --write-sub --sub-lang en --skip-download --output "Listening_Practice_Indian_Accent_Audio32" https://www.youtube.com/watch?v=YDp3EE0AM0s
0


FileNotFoundError: [Errno 2] No such file or directory: '/Users/anshul/Desktop/ISTEM/azure-speech-testing/Listening_Practice_Indian_Accent_Audio32.en.vtt'

In [24]:
import os
video_title = ''.join(youtube.xpath("//span[@id='eow-title']/@title")) #downloading the title from url
video_title=(re.sub(' +', ' ',re.sub(r'[^\w\s]','',video_title) )).replace(' ','_')
audio='youtube-dl --extract-audio --audio-format mp3 --output "'+video_title+'.mp3" '+url
os.system(audio)


0

In [18]:
from pytube import YouTube 
  
#where to save 
SAVE_PATH = '/Users/anshul/Desktop/ISTEM/azure-speech-testin/videos/' #to_do 
  
#link of the video to be downloaded 
link="https://www.youtube.com/watch?v=kcW4ABcY3zI"
yt = YouTube(link) 
try: 
    #object creation using YouTube which was imported in the beginning 
    yt = YouTube(link) 
except: 
    print("Connection Error") #to handle exception 
  
#filters out all the files with "mp4" extension 
mp4files = yt.filter('mp4') 
  
yt.set_filename('GeeksforGeeks Video') #to set the name of the file 
  
#get the video with the extension and resolution passed in the get() function 
d_video = yt.get(mp4files[-1].extension,mp4files[-1].resolution) 
try: 
    #downloading the video 
    d_video.download(SAVE_PATH) 
except: 
    print("Some Error!") 
print('Task Completed!') 

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1056)>